Firstly, we need to upload the raw battery xlsx spreadsheet. Then we read it into a dataframe (b). Then plot out by hubid.

In [ ]:
!ls
!pwd
try:
    from google.colab import files
    uploaded = files.upload()
except:
    pass # failed to load module on laptop

In [ ]:
import pandas as pd
import dateutil.parser as du
from datetime import datetime as dt

full_set = pd.read_excel ('/Users/tim/Projects/iotaa/battery/battery analysis.xlsx')


Remove data from before the last HubV2 update as the quality is unknown before then.

In [ ]:
b= full_set[full_set ['Date Time'] > du.parse ("2019-5-15")]
#b['ts']=b.iloc[:,3].apply (lambda x: dt.utcfromtimestamp (x/1000)) # irrelevant as 'Date Time' == 'Epoch TS'
b['ts'] = b.loc [:, 'Date Time']

In [ ]:
#b.iloc[:,[2, 5,6]]

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

%matplotlib inline

def plotting (axs, figs, rows, cols):
    count=0
    r,c = 0, 0
    fig = 0
    hubs = []
    axi = axs[fig]
#fig, ax = plt.subplots (nrows=8, ncols=4, sharex=True , figsize = (18,32))
    #fig, ax = plt.subplots (nrows=8, ncols=4, sharex=True , figsize = (20, 12))

#for h, d in b.groupby ('Hub Mac'):
    for h, d in b.groupby ('ClientID'):
  #if '5d25' not in h:
  #  continue
        if cols==1:  # non-orthogonal api :-(    
            ax = axi[r]
        else:
            ax = axi[r,c]
        ax.set_title ("Hub: {}".format (h))
        print (h)
        hubs.append (h)
        box = ax.get_position()
        ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

        loc=mdates.DayLocator(interval=56)
        timfmt = mdates.AutoDateFormatter(loc)
        ax.xaxis.set_major_formatter (timfmt)
        ax.xaxis.set_major_locator(loc)
        ax.xaxis.set_tick_params(rotation=30)


        for v, g in d.groupby ('Device Slot ID + Node'):
#    ax.scatter (x=g['ts'], y=g['Battery'], alpha=0.3, label=v[0:5])
            ax.plot_date (x=g['ts'], y=g['Battery'], alpha=0.3, label=v) #[0:8])
    # Put a legend to the right of the current axis
            print (v)
            ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#    ax.legend()

#  ax[count].xaxis.grid(True)
        r += 1
        if r >= rows:
            c += 1
            if c >= cols:
                c = 0
                fig += 1
                if fig < figs:
                    ax = axs[fig]
            r = 0
    return hubs

figs = 1
rows, cols = 11, 1
ax=[]
for i in range (figs):
    fig, axes = plt.subplots (nrows=rows, ncols=cols, sharex=True, sharey=True, figsize = (20, 35))
    ax.append (axes)

hubs = plotting (ax, figs, rows, cols)
with open ('hubs', 'w') as of:
    for h in hubs:
        of.write ("{}\n".format (h))

In [ ]:
len ([x[1][1] for x in b.iterrows () if '5b61' in x[1][1]])

In [ ]:
len ([x[1][1] for x in b.iterrows () if '5b61' not in x[1][1]])

In [ ]:
[(x[1][1], x[1][0]) for x in b.iterrows () if '5b61' in x[1][1]]

In [ ]:
b.columns

In [ ]:
len (b[b['Date Time'] > du.parse ("2019-5-15")] )

In [ ]:
min (b['ts'])

In [ ]:
print (hubs)

In [ ]:
ls=b[b['ClientID'] == ' 5d3ea83ac7fe0554e22ae04c']

In [ ]:
set (ls['Device Slot ID + Node'])

In [ ]:
ls=b

In [ ]:
len ([(x, min (ls[ls['Device Slot ID + Node']==x]['Battery'])) for x in set (ls ['Device Slot ID + Node'])])

In [ ]:
low = [(x, min (ls[ls['Device Slot ID + Node']==x]['Battery'])) for x in set (ls ['Device Slot ID + Node']) if min (ls[ls['Device Slot ID + Node']==x]['Battery']) < 100]

In [ ]:
len (low)

In [ ]:
for did in [x[0] for x in low]:
    print (did)
    print (b[b['Device Slot ID + Node'] == did][['Battery', 'ts']])

In [ ]:
b[b['Device Slot ID + Node'] == low [-4][0]]

C's hub changed at the end of July. Therefore are all of her devices before that day should be ignored.

A on 31/7, 3/9 B.